In [2]:
import tensorflow.keras as keras
import multiprocessing
import zipfile

ImportError: cannot import name 'abs'

In [ ]:
import numpy as np
import pandas as pd
from PIL import Image

In [ ]:
submissions = pd.read_csv('instance/uploads/data.csv')

In [ ]:
# DATASET_PATH = 'dogmentation_val.zip'
DATASET_IMAGE_COLUMN = 'image'
DATASET_MASK_COLUMN = 'mask'
BATCH_SIZE = 8

def load_image(path: str) -> Image:
    image = Image.open(path)
    image.load()
    return image

dataset = pd.read_csv('test/index.csv')
dataset[DATASET_IMAGE_COLUMN] = dataset[DATASET_IMAGE_COLUMN].apply(
    lambda path: load_image('test/'+path)
)
dataset[DATASET_MASK_COLUMN] = dataset[DATASET_MASK_COLUMN].apply(
    lambda path: load_image('test/'+path)
)

def test_model(data, model_filename, is_bgr):
    """Test model at `model_filename` on PIL Images in `data`, return FP32 array."""
    # Data is list of {'image': encoded_image}
    model_path = to_uploads(model_filename)

    if isinstance(data.get('rows')[0].get('image'), str):
        test_images = [decode_image(x['image']) for x in data['rows']]
    elif isinstance(data.get('rows')[0].get('image'), Image.Image):
        # Is already encoded
        test_images = [x['image'] for x in data['rows']]
    test_images = np.stack(map(np.array, test_images))
    test_images = test_images.astype(np.float32) / 255

    if is_bgr:
        test_images = test_images[..., ::-1]  # BGR

    # Get scaled predictions
    with multiprocessing.Pool() as pool:
        predictions = pool.starmap(model_inference,
                                   [(model_path, test_images)])[0]

    return predictions

def evaluate_test_dataset(model_filename, is_bgr=True):
    """Return list of encoded images `rows` with responses for test `dataset`."""
    dataset['encoded'] = dataset[DATASET_IMAGE_COLUMN].apply(encode_image)

    rows = []

    for i in range(0, len(dataset), BATCH_SIZE):
        data = {
            'rows': [{
                'image': encoded
            } for encoded in dataset['encoded'].iloc[i:(i + BATCH_SIZE)]]
        }

        predictions_arr = test_model(data, model_filename, is_bgr)
        rows.extend(predictions_arr)        
    return rows

def model_inference(model_path, test_images):

    keras.backend.clear_session()
    predictions = []
    # with tf.Session(graph=tf.Graph()) as sess:
    #     K.set_session(sess)
    model = keras.models.load_model(model_path, compile=False)
    model.compile(
        loss=keras.losses.binary_crossentropy,
        optimizer=keras.optimizers.Adam())
    predictions = model.predict(test_images)
    return predictions

def compute_iou(test_results_list, field_out):
    """Compute IoU for `test_results_list`."""
    try:
        # TODO: Handle API calls and catch errors
        test = test_results_list[0][field_out]
        predicted_mask = np.stack(
            np.array(decode_image(row[field_out])).reshape(128, 128)
            for row in test_results_list)
    except IndexError:
        predicted_mask = np.stack(
            (pred).reshape(128, 128) for pred in test_results_list)

    predicted_mask = predicted_mask > 0.5
    img_count = len(predicted_mask)
    true_mask = np.stack(
        np.array(mask) for mask in dataset[DATASET_MASK_COLUMN][:img_count])
    true_mask = true_mask > 255 / 2

    intersection = np.logical_and(predicted_mask, true_mask)
    union = np.logical_or(predicted_mask, true_mask)
    iou_score = np.sum(intersection) / np.sum(union)

    print('IoU', iou_score)
    return iou_score

import io, base64
def encode_image(image: Image):
    """Convert PIL Image `image` to base64 encoded image."""
    image_bytes = io.BytesIO()
    image.save(image_bytes, format='png')
    return 'data:image/png;base64,' + base64.b64encode(
        image_bytes.getvalue()).decode()

def decode_image(data_uri: str):
    """Convert `data_uri` to PIL Image."""
    format_str, image_str = data_uri.split(',', 1)
    if format_str != 'data:image/png;base64':
        raise ValueError('Format not supported')

    with io.BytesIO(base64.b64decode(image_str)) as image_bytes:
        image = Image.open(image_bytes)
        image.load()
        return image
def to_uploads(path):
    return 'uploads/'+path

In [ ]:
submissions.iloc[3][['name']].values[0]

In [ ]:
for idx, filename in enumerate(submissions.url):
    test_results_list  = evaluate_test_dataset(model_filename=filename)
    out_img_filename = get_sample_overlay(test_results_list)
    iou = compute_iou(test_results_list, 'mask')
    print(submissions.iloc[idx][['name']].values[0], iou)